In [1]:
# Import necessary libraries
import json
import pandas as pd
import openai
import elasticsearch
from groq import Groq
from dotenv import load_dotenv
import os
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.auto import tqdm

In [2]:
# Load environment variables
load_dotenv()

True

In [3]:
# Setup project paths

#change the below base_fo,der
base_folder = 'D:/Projects/datatalks-final-project/'
input_data_folder = base_folder + 'dataset/'


In [4]:
# Get the current working directory (optional step)
os.getcwd()

'd:\\Projects\\datatalks-final-project\\dataset\\gold_standard_data'

In [5]:
# Setup the OpenAI client to use either Groq, OpenAI.com, or Ollama API
load_dotenv(override=True)
API_HOST = os.getenv("API_HOST")
API_HOST

'groq'

In [6]:
if API_HOST == "groq":
    client = client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
    MODEL_NAME = os.getenv("GROQ_MODEL")

elif API_HOST == "ollama":
    client = openai.OpenAI(
        base_url=os.getenv("OLLAMA_ENDPOINT"),
        api_key="nokeyneeded",
    )
    MODEL_NAME = os.getenv("OLLAMA_MODEL")

elif API_HOST == "github":
    client = openai.OpenAI(base_url="https://models.inference.ai.azure.com", api_key=os.getenv("GITHUB_TOKEN"))
    MODEL_NAME = os.getenv("GITHUB_MODEL")

else:
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    MODEL_NAME = os.getenv("OPENAI_MODEL")

In [7]:
client

In [8]:
MODEL_NAME

'llama3-8b-8192'

In [18]:
with open(input_data_folder + 'main_faq_database.json', 'rt') as f_in:
    data = json.load(f_in)

In [19]:
documents = []
for dish in data['dishes']:
    dish_name = dish['dish name']
    for doc in dish['documents']:
        doc['dish_name'] = dish_name  # Add dish_name to each document
        documents.append(doc)

In [20]:
# The prompt template for generating questions
prompt_template = """
You are emulating a user who's ordering food from a restaurant.
Based on the following information about the dish, generate 5 questions a customer might ask which have similar meanings.
Use as few words from the dish description as possible while making the questions which have similar meanings.

Make sure to include the dish name in the questions.

Don't start with "Here are the 5 questions a customer might ask: or anything similar. Just give straight away questions.
section: {section}
question: {question} + {dish_name}
answer: {text}

Provide the output in JSON format:

["question1", "question2", "question3", "question4", "question5"]

""".strip()

In [21]:
# Function to generate questions based on the document
def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    
    try:
        # Call Groq API to generate the response
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Print the response to debug
        print("Response from API:", response)
        
        # Extract the content of the response
        content = response.choices[0].message.content
        
        # Use regex to extract the list items between square brackets
        match = re.search(r'\[(.*?)\]', content, re.DOTALL)
        if match:
            # Split the content between commas and clean up the formatting
            generated_questions = [item.strip().strip('"') for item in match.group(1).split(',')]
        else:
            print(f"No questions found for doc {doc['id']}")
            generated_questions = []
    
    except Exception as e:
        print(f"Error during API call for doc {doc['id']}: {e}")
        generated_questions = []
    
    return generated_questions


In [22]:
sample_documents = documents [2544:]
sample_documents # Modify the number of samples as needed


[{'id': '26014_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The mexican tres leche cake contains the following ingredients: ['all-purpose flour', 'baking powder', 'sugar', 'unsalted butter', 'eggs', 'milk', 'sweetened condensed milk', 'evaporated milk', 'rum', 'heavy cream', 'vanilla extract']",
  'dish_name': 'mexican tres leche cake'},
 {'id': '26014_2',
  'question': 'How many calories does it have?',
  'section': 'calories',
  'text': 'The mexican tres leche cake has 630.0 calories.',
  'dish_name': 'mexican tres leche cake'},
 {'id': '26014_3',
  'question': 'How much total fat does it contain?',
  'section': 'nutritional',
  'text': 'The mexican tres leche cake has N/A grams of total fat (PDV).',
  'dish_name': 'mexican tres leche cake'},
 {'id': '26014_4',
  'question': 'What is the sugar content?',
  'section': 'nutritional',
  'text': 'The mexican tres leche cake has N/A grams of sugar (PDV).',
  'dish_name': 'mexican tres leche cake'},

In [64]:
# Define the target dish name
target_dish = "Mexican Tres Leche Cake".lower()

# Find the index where the dish_name matches
for idx, doc in enumerate(documents):
    if doc['dish_name'] == target_dish:
        print(f"The dish '{target_dish}' starts at index {idx}")
        break
else:
    print(f"The dish '{target_dish}' was not found.")


The dish 'mexican tres leche cake' starts at index 2544


In [69]:
# Store the results
results = {}
sample_documents = documents[2544:]  # Adjust the number of samples as needed

In [72]:
import re
import time

# Loop through each document and generate questions
for doc in tqdm(sample_documents):
    doc_id = doc['id']
    if doc_id in results:
        continue
    
    # Generate questions
    questions = generate_questions(doc)
    results[doc_id] = questions
    
    # Add a delay to avoid rate limits
    time.sleep(1)  # Wait 2 seconds before making the next API call

# Check the results
print(results)


  0%|          | 0/1040 [00:00<?, ?it/s]

Response from API: ChatCompletion(id='chatcmpl-b7a23cea-d0a8-4085-a6a8-ac64b64a7427', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='["How many calories does the Mexican Tres Leche Cake contain?", "What\'s the calorie count of the Mexican Tres Leche Cake?", "Can you tell me the calorie amount of Mexican Tres Leche Cake?", "How many calories did they use to make the Mexican Tres Leche Cake?", "What\'s the estimated calorie content of the Mexican Tres Leche Cake?"]', role='assistant', function_call=None, tool_calls=None))], created=1725680948, model='llama3-8b-8192', object='chat.completion', system_fingerprint='fp_179b0f92c9', usage=CompletionUsage(completion_tokens=80, prompt_tokens=166, total_tokens=246, completion_time=0.066666667, prompt_time=0.035150708, queue_time=0.5780751430000001, total_time=0.101817375), x_groq={'id': 'req_01j759k54pf7xv16dmpd03rxf9'})
Response from API: ChatCompletion(id='chatcmpl-b8d3ad68-d3e0-4d13-8a57-f

In [75]:
results

{'26014_1': ['How is the Mexican Tres Leche Cake made?',
  'What goes into baking the Mexican Tres Leche Cake?',
  'What are the main components of the Mexican Tres Leche Cake?',
  'Can you reveal the recipe for the Mexican Tres Leche Cake?',
  "What's the secret to the taste of the Mexican Tres Leche Cake?"],
 '26014_2': ['How many calories does the Mexican Tres Leche Cake contain?',
  "What's the calorie count of the Mexican Tres Leche Cake?",
  'Can you tell me the calorie amount of Mexican Tres Leche Cake?',
  'How many calories did they use to make the Mexican Tres Leche Cake?',
  "What's the estimated calorie content of the Mexican Tres Leche Cake?"],
 '26014_3': ['Mexican tres leche cake',
  'Is it very oily?',
  'Mexican tres leche cake',
  'How fatty is it?',
  'Mexican tres leche cake',
  'Are there a lot of fats in this?',
  'Mexican tres leche cake',
  'Does it have a lot of grams?'],
 '26014_4': ['Is the sugar content high in the Mexican tres leche cake?',
  'Is there a lo

In [76]:
doc_index = {d['id']: d for d in documents}


In [77]:
# Define the course and document IDs
course_name = 'food-gold-data'
dish_name = doc_index[doc_id]['dish_name']

# Prepare data for writing to CSV
csv_data = []
for doc_id, questions in results.items():
    for question in questions:
        csv_data.append([question, dish_name, doc_id])


In [78]:
ground_df = pd.DataFrame(csv_data, columns=['question', 'course', 'document'])

In [79]:
ground_df.head()

,question,course,document
0,How is the Mexican Tres Leche Cake made?,new jersey crumb cake,26014_1
1,What goes into baking the Mexican Tres Leche C...,new jersey crumb cake,26014_1
2,What are the main components of the Mexican Tr...,new jersey crumb cake,26014_1
3,Can you reveal the recipe for the Mexican Tres...,new jersey crumb cake,26014_1
4,What's the secret to the taste of the Mexican ...,new jersey crumb cake,26014_1


In [80]:
# Write the CSV file
ground_df.to_csv('ground-truth-data.csv', index=False)